

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

- Anthony Olcek
- N18364039

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [ ]:
# %%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-deps "xformers<0.0.26" "trl<0.9.0" "peft<0.12.0" "accelerate<0.32.0" "bitsandbytes<0.44.0" "transformers<4.43.0"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ybhie32p/unsloth_a60dc76f01644271881d4a8576f06540
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ybhie32p/unsloth_a60dc76f01644271881d4a8576f06540
  Resolved https://github.com/unslothai/unsloth.git to commit 2267b5c5532957141a33bfa5bb9f0b220a4b3efe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 94.5 MB/s eta 0:00:00


In [ ]:
!pip install unsloth_zoo

## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [ ]:
from unsloth import FastLanguageModel
import torch

# Load Llama-3.1-8B model and tokenizer (optimized)
max_seq_length = 2048          # Supports long reasoning chains
compress_pos_emb = 2           # Halves memory cost for long sequences
dtype = None                   # Auto-detects best type (bf16 or fp16)
load_in_4bit = True            # Enables 4-bit quantization for Colab GPUs

# Load the base model and tokenizer using Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",  # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Enable gradient checkpointing to reduce memory usage
try:
    model.gradient_checkpointing_enable()
    print("Gradient checkpointing enabled.")
except Exception as e:
    print(f"Gradient checkpointing not available: {e}")

# Check model device and dtype
print(f"Model dtype: {next(model.parameters()).dtype}")
print(f"Device: {next(model.parameters()).device}")


ModuleNotFoundError: No module named 'unsloth'

## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **5,000 samples for training** and **500 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [ ]:
from datasets import load_dataset
from sklearn.model_selection import KFold

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for reproducibility
shuffled_dataset = full_dataset.shuffle(seed=42)

# Define subset sizes for faster local or Colab runs
NUM_TRAIN = 5000
NUM_VALID = 500

# Create a smaller subset for training and validation
train_dataset = shuffled_dataset.select(range(NUM_TRAIN))
validation_dataset = shuffled_dataset.select(range(NUM_TRAIN, NUM_TRAIN + NUM_VALID))

# Set up K-Fold cross-validation
N_FOLDS = 5
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

# Precompute fold indices
fold_indices = list(kf.split(range(NUM_TRAIN + NUM_VALID)))

# Use the first fold as an example (loop later for full CV)
train_idx, val_idx = fold_indices[0]

cv_train_dataset = shuffled_dataset.select(train_idx.tolist())
cv_valid_dataset = shuffled_dataset.select(val_idx.tolist())

print(f"Fold 1 | Training samples: {len(cv_train_dataset)} | Validation samples: {len(cv_valid_dataset)}")


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Fold 1 | Training samples: 4400 | Validation samples: 1100


In [ ]:
# Instructional prompt template for training
training_prompt = """You are an expert mathematician evaluating whether a given solution correctly answers a math question. Follow this rigorous process:

1. Parse the Question Carefully: Read the question word by word. Identify all variables, constants, constraints, and exactly what is being asked.

2. Analyze the Provided Solution: Examine the solution step by step. Check for:
   - Correct interpretation of the question
   - Proper mathematical operations and formulas
   - Logical reasoning flow
   - Computational accuracy
   - Appropriate units and rounding

3. Solve Independently: Work through the problem yourself step by step without looking at the provided solution.

4. Cross-Verify: Compare your independent solution with the provided solution. Check if they arrive at the same final answer through valid methods.

5. Validate Logic: Ensure the solution actually answers what was asked in the question, not a similar or related question.

6. Final Judgment: Only if the provided solution is completely correct in both method and final answer should you respond with 'True'. Any errors in reasoning, calculation, or interpretation warrant 'False'.

After completing this analysis, respond with ONLY a single word: 'True' or 'False'.

Question:
{}
Solution:
{}
Output:"""

# Retrieve model End-of-Sequence token
EOS_TOKEN = tokenizer.eos_token

# Format each dataset entry into the training prompt
def formatting_prompts_func(examples):
    texts = []
    for i, (q, s, o) in enumerate(zip(examples["question"], examples["solution"], examples["is_correct"])):
        # Add small sliding context of previous questions
        context_window = 2
        prev_q = " ".join(examples["question"][max(0, i - context_window):i])
        formatted = training_prompt.format(prev_q + " " + q.strip(), str(s).strip(), str(o).strip())
        formatted += tokenizer.eos_token
        texts.append(formatted)
    return {"text": texts}


# Apply formatting to main datasets
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True)

# Filter out samples exceeding 2048 tokens
formatted_train_dataset = formatted_train_dataset.filter(
    lambda ex: len(tokenizer(ex["text"])["input_ids"]) <= 2048
)
formatted_validation_dataset = formatted_validation_dataset.filter(
    lambda ex: len(tokenizer(ex["text"])["input_ids"]) <= 2048
)

# Format cross-validation datasets if used
formatted_cv_train = cv_train_dataset.map(formatting_prompts_func, batched=True)
formatted_cv_valid = cv_valid_dataset.map(formatting_prompts_func, batched=True)

print("Datasets formatted successfully and ready for tokenization.")


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

Datasets formatted successfully and ready for tokenization.


## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory. We'll use a small **rank** (`r = 8`) to keep the training process light and quick for this starter notebook.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,  # LoRA rank; controls the adapter's capacity for learning new representations
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,      # scaling factor (commonly set to ~2 × r)
    lora_dropout = 0.05,  # dropout applied to LoRA layers to improve generalization
    bias = "none",        # avoids bias injection; keeps model stable
    use_gradient_checkpointing = "unsloth",  # memory-efficient training for long sequences
    random_state = 42,    # ensures reproducibility across runs
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.12 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

We will train for just **one epoch** (a single pass over our 5,000-sample dataset) to keep this demonstration fast.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback

# Format validation dataset for evaluation and early stopping
formatted_val_dataset = validation_dataset.map(formatting_prompts_func, batched=True)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    eval_dataset = formatted_val_dataset,        # add validation set
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.05,               # smoother ramp-up
        max_grad_norm = 1.0,               # gradient clipping
        max_steps = 200,                   # more training iterations
        learning_rate = 1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "paged_adamw_8bit",       # memory-efficient optimizer
        weight_decay = 0.01,
        lr_scheduler_type = "cosine_with_restarts",   # enables warm restarts
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
        save_total_limit = 2,             # keep only latest checkpoints

        # Compatible argument names for transformers <4.43
        eval_strategy = "steps",          # evaluate periodically
        eval_steps = 20,                  # run validation every 20 steps
        load_best_model_at_end = True,    # reload best checkpoint automatically
    ),
)

# Add early stopping (stop if no improvement for 2 consecutive evaluations)
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/4999 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕


In [ ]:
import torch
import gc
import numpy as np
import itertools
from sklearn.model_selection import KFold

N_FOLDS = 5
RANDOM_SEED = 42

param_grid = {
    "learning_rate": [5e-5, 1e-4, 2e-4],
    "lora_r": [4, 8],
    "lora_alpha": [8, 16],
}

# Preprocess dataset once
print("Preprocessing dataset...")
formatted_dataset = shuffled_dataset.map(formatting_prompts_func, batched=True)
formatted_dataset = formatted_dataset.filter(
    lambda ex: len(tokenizer(ex["text"])["input_ids"]) <= 2048
)
print(f"Final dataset size: {len(formatted_dataset)} examples")

kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)
fold_results = []
ensemble_paths = []

for fold, (train_idx, val_idx) in enumerate(kf.split(formatted_dataset)):
    print(f"\nFold {fold + 1}/{N_FOLDS}")

    train_fold = formatted_dataset.select(train_idx.tolist())
    val_fold = formatted_dataset.select(val_idx.tolist())

    best_val_loss = float("inf")
    best_config = None
    best_model_path = f"model_fold_{fold}.pt"

    # Loop through hyperparameter combinations
    for lr, r, alpha in itertools.product(
        param_grid["learning_rate"], param_grid["lora_r"], param_grid["lora_alpha"]
    ):
        print(f"\nFold {fold + 1} | lr={lr}, r={r}, alpha={alpha}")

        tuned_model = FastLanguageModel.get_peft_model(
            model,
            r=r,
            target_modules=[
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj"
            ],
            lora_alpha=alpha,
            lora_dropout=0.05,
            bias="none",
            use_gradient_checkpointing="unsloth",
            random_state=RANDOM_SEED,
        )

        trainer.args.learning_rate = lr
        trainer.train_dataset = train_fold
        trainer.eval_dataset = val_fold

        trainer.train()
        metrics = trainer.evaluate()
        val_loss = metrics.get("eval_loss", None)

        if val_loss is not None and val_loss < best_val_loss:
            best_val_loss = val_loss
            best_config = {"lr": lr, "r": r, "alpha": alpha}
            torch.save(tuned_model.state_dict(), best_model_path)
            print(f"New best model saved for fold {fold + 1} (loss {val_loss:.4f})")

        # Release memory
        del tuned_model
        torch.cuda.empty_cache()
        gc.collect()

    fold_results.append({
        "fold": fold + 1,
        "best_val_loss": best_val_loss,
        "best_config": best_config
    })
    ensemble_paths.append(best_model_path)

    torch.cuda.empty_cache()
    gc.collect()

# Compute summary statistics
mean_loss = np.mean([f["best_val_loss"] for f in fold_results])
std_loss = np.std([f["best_val_loss"] for f in fold_results])
print(f"\nCross-validation complete. Mean eval_loss = {mean_loss:.4f} ± {std_loss:.4f}")

for res in fold_results:
    print(f"Fold {res['fold']}: loss={res['best_val_loss']:.4f}, config={res['best_config']}")

print("\nSaved model checkpoints:", ensemble_paths)


Preprocessing dataset...


NameError: name 'shuffled_dataset' is not defined

In [ ]:
from torch.nn.functional import cosine_similarity

# Function to compute embedding similarity between question and solution
def compute_similarity(question, solution):
    q_tokens = tokenizer(question, return_tensors="pt", truncation=True, max_length=2048).input_ids
    s_tokens = tokenizer(solution, return_tensors="pt", truncation=True, max_length=2048).input_ids

    with torch.no_grad():
        q_emb = model.get_input_embeddings()(q_tokens)
        s_emb = model.get_input_embeddings()(s_tokens)

    similarity = cosine_similarity(q_emb.mean(dim=1), s_emb.mean(dim=1)).item()
    return similarity

# Example similarity check on one validation sample
sample = validation_dataset[0]
sim_score = compute_similarity(sample["question"], sample["solution"])
print(f"Question–Solution embedding similarity: {sim_score:.3f}")

# Optional: log similarity scores for all validation samples to analyze correlation with correctness
similarity_scores = []
for i, ex in enumerate(validation_dataset):
    sim = compute_similarity(ex["question"], ex["solution"])
    similarity_scores.append({
        "index": i,
        "similarity": sim,
        "is_correct": ex["is_correct"]
    })

# Compute and display average similarity grouped by correctness
import pandas as pd
sim_df = pd.DataFrame(similarity_scores)
mean_true = sim_df[sim_df["is_correct"] == True]["similarity"].mean()
mean_false = sim_df[sim_df["is_correct"] == False]["similarity"].mean()

print(f"\nAverage similarity for correct solutions: {mean_true:.3f}")
print(f"Average similarity for incorrect solutions: {mean_false:.3f}")

# Save similarity logs for analysis if needed
sim_df.to_csv("embedding_similarity_log.csv", index=False)
print("\nSaved similarity analysis to embedding_similarity_log.csv")



## **Step 6: Inference and Evaluation**

Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [ ]:
from torch.nn.functional import softmax

# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are an expert mathematician evaluating whether a given solution correctly answers a math question. Follow this rigorous process:

1. **Parse the Question Carefully**: Read the question word by word. Identify all variables, constants, constraints, and exactly what is being asked.

2. **Analyze the Provided Solution**: Examine the solution step by step. Check for:
   - Correct interpretation of the question
   - Proper mathematical operations and formulas
   - Logical reasoning flow
   - Computational accuracy
   - Appropriate units and rounding

3. **Solve Independently**: Work through the problem yourself step by step without looking at the provided solution. Show your work clearly.

4. **Cross-Verify**: Compare your independent solution with the provided solution. Check if they arrive at the same final answer through valid methods.

5. **Validate Logic**: Ensure the solution actually answers what was asked in the question, not a similar or related question.

6. **Final Judgment**: Only if the provided solution is completely correct in both method and final answer should you respond with 'True'. Any errors in reasoning, calculation, or interpretation warrant 'False'.

After completing this analysis, respond with ONLY a single word: 'True' or 'False'.

Question:
{}
Solution:
{}
Output:"""

# Select a sample from the validation set (change index as needed)
example = validation_dataset[10]
question = example["question"]
solution = example["solution"]

# Format the prompt with the validation data
inputs = tokenizer(
    [inference_prompt.format(question, str(solution))],
    return_tensors="pt"
).to("cuda")

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
response_text = tokenizer.batch_decode(outputs)[0]

# Optional: compute confidence of the prediction
logits = model(**inputs).logits[:, -1, :]
probs = softmax(logits, dim=-1)
confidence = float(probs.max())

# Parse prediction text
try:
    prediction = response_text.split("Output:")[1].strip().split()[0]
except IndexError:
    prediction = "UNKNOWN"

# Display results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
print(prediction)
print(f"\n#### CONFIDENCE ####\n{confidence:.3f}")
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [ ]:
# Generate Submission File with Ensemble Averaging Across Folds

import pandas as pd
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
import torch

# Load the official test dataset
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")

# Function to parse "True" or "False" from model output
def parse_output(response_text):
    if "Output:" in response_text:
        output_part = response_text.split("Output:")[-1]
    else:
        output_part = response_text
    text_lower = output_part.strip().lower()
    if "true" in text_lower:
        return True
    elif "false" in text_lower:
        return False
    return False  # Default fallback

# Function to run inference on the test set using a given model
def run_inference_on_test_set(model):
    preds = []
    for example in tqdm(test_dataset, desc="Generating predictions"):
        question = example["question"]
        solution = example["solution"]
        prompt = inference_prompt.format(question, str(solution))

        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
        response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        preds.append(parse_output(response_text))
    return np.array(preds, dtype=float)

# Ensemble predictions across folds
predictions_folds = []

print("\nRunning ensemble inference across folds...")
for fold in range(N_FOLDS):
    model_path = f"model_fold_{fold}.pt"
    print(f"Loading model from {model_path}...")

    # Reload base model and LoRA adapters
    fold_model = FastLanguageModel.from_pretrained(
        model_name="unsloth/Meta-Llama-3.1-8B",
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )[0]
    fold_model = FastLanguageModel.get_peft_model(
        fold_model,
        r=8,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj"],
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=42,
    )

    fold_model.load_state_dict(torch.load(model_path))
    fold_model = fold_model.to("cuda").eval()

    preds = run_inference_on_test_set(fold_model)
    predictions_folds.append(preds)
    torch.cuda.empty_cache()

# Average ensemble predictions across folds
final_preds = (np.mean(predictions_folds, axis=0) > 0.5).astype(bool)

# Create submission DataFrame
submission = pd.DataFrame({
    "ID": range(len(final_preds)),
    "is_correct": final_preds
})

# Save ensemble submission
submission.to_csv("submission_ensemble.csv", index=False)

print("\nEnsemble submission file 'submission_ensemble.csv' created successfully.")
print("You can now download and submit it to the competition leaderboard.")


# SAVE THE MODEL TO DRIVE AND RUN INFERENCE
Add code to save the model checkpoint to Google Drive, load the model from the checkpoint, and generate the final submission CSV file.

## Mount google drive

### Subtask:
Mount Google Drive to save the model checkpoint.


**Reasoning**:
Mount Google Drive to save the model checkpoint.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Save model checkpoint

### Subtask:
Save the trained model checkpoint to the specified path in Google Drive.


**Reasoning**:
Define the save path and save the model and tokenizer to Google Drive.



In [ ]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

## Load model from checkpoint

### Subtask:
Load the model from the saved checkpoint.


**Reasoning**:
Load the model and tokenizer from the saved checkpoint path in Google Drive and prepare the model for inference.



In [ ]:
# Define the path where the model checkpoint was saved in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Load the model and tokenizer from the saved path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

print(f"Model and tokenizer loaded from: {save_path}")

## Generate submission file

### Subtask:
Generate the submission CSV file using the loaded model.


**Reasoning**:
Generate the submission CSV file by iterating through the test dataset, generating predictions using the loaded model, and saving the results to a pandas DataFrame.



In [ ]:
# Generate predictions for the official test dataset and create submission.csv

import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Create the prompt template for inference (no answer included)
inference_prompt = """You are an expert mathematician evaluating whether a given solution correctly answers a math question. Follow this rigorous process:

1. **Parse the Question Carefully**: Read the question word by word. Identify all variables, constants, constraints, and exactly what is being asked.

2. **Analyze the Provided Solution**: Examine the solution step by step. Check for:
   - Correct interpretation of the question
   - Proper mathematical operations and formulas
   - Logical reasoning flow
   - Computational accuracy
   - Appropriate units and rounding

3. **Solve Independently**: Work through the problem yourself step by step without looking at the provided solution. Show your work clearly.

4. **Cross-Verify**: Compare your independent solution with the provided solution. Check if they arrive at the same final answer through valid methods.

5. **Validate Logic**: Ensure the solution actually answers what was asked in the question, not a similar or related question.

6. **Final Judgment**: Only if the provided solution is completely correct in both method and final answer should you respond with 'True'. Any errors in reasoning, calculation, or interpretation warrant 'False'.

After completing this analysis, respond with ONLY a single word: 'True' or 'False'.

Question:
{}
Solution:
{}
Output:"""

# Function to parse the model's generated output into a boolean
def parse_output(response_text):
    # Extract portion after "Output:" if present
    if "Output:" in response_text:
        output_part = response_text.split("Output:")[-1]
    else:
        output_part = response_text
    # Normalize text for comparison
    text_lower = output_part.strip().lower()
    if "true" in text_lower:
        return True
    elif "false" in text_lower:
        return False
    return False  # Default fallback if unclear

# Loop through test dataset
for example in tqdm(test_dataset, desc="Generating predictions"):
    question = example["question"]
    solution = example["solution"]

    # Construct inference prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate model response
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Parse and store prediction
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create submission DataFrame
submission = pd.DataFrame({
    "ID": range(len(predictions)),
    "is_correct": predictions
})

# Save predictions to CSV
submission.to_csv("submission.csv", index=False)

print("\nSubmission file 'submission.csv' created successfully.")
print("You can now download this file and submit it to the competition.")